In [1]:
import minsearch 
import json
from pprint import pprint 





In [4]:
from dotenv import load_dotenv
import openai
import os
from openai import OpenAI
load_dotenv()  # defaults to .env in the same directory

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": "tell me a joke"}]
)
print(response.choices[0].message.content)





Why don't skeletons fight each other?

They don't have the guts!


In [140]:
# Q1
!curl http://localhost:9200

{
  "name" : "35bc0b180d8a",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "_eKjvkP7QROgLJfiRInGnw",
  "version" : {
    "number" : "8.17.6",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "dbcbbbd0bc4924cfeb28929dc05d82d662c527b7",
    "build_date" : "2025-04-30T14:07:12.231372970Z",
    "build_snapshot" : false,
    "lucene_version" : "9.12.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [64]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [141]:

[type(documents), len(documents)]
documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

In [85]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")
es.info()

ObjectApiResponse({'name': '35bc0b180d8a', 'cluster_name': 'docker-cluster', 'cluster_uuid': '_eKjvkP7QROgLJfiRInGnw', 'version': {'number': '8.17.6', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'dbcbbbd0bc4924cfeb28929dc05d82d662c527b7', 'build_date': '2025-04-30T14:07:12.231372970Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [87]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"
response = es.indices.create(index=index_name, body=index_settings)

response

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [88]:
# Q2
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [ ]:
user_question= "How do execute a command on a Kubernetes pod?"
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": user_question,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },  
        }
    }
}

response = es.search(index=index_name, body=search_query)


In [96]:
# Q3
# Print the keys at the top level
print("Top-level keys in response:")
print(response.keys())

# Pretty print the whole response (be careful if it's very large)
print("\nFull response structure:")
pprint(response)

# To explore further, for example, the hits:
if "hits" in response:
    print("\nKeys in response['hits']:")
    print(response["hits"].keys())
    print("\nFirst hit example:")
    if response["hits"]["hits"]:
        pprint(response["hits"]["hits"][0])



Top-level keys in response:
dict_keys(['took', 'timed_out', '_shards', 'hits'])

Full response structure:
ObjectApiResponse({'took': 23, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 739, 'relation': 'eq'}, 'max_score': 44.50556, 'hits': [{'_index': 'course-questions', '_id': 'lvSZHZcBdgauo6t84jmq', '_score': 44.50556, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}, {'_index': 'course-questions', '_id': 'JfSZHZcBdgauo6t84zrf', '_score': 35.433445, '_source': {'text': 'Deploy and Acces

In [99]:
user_question= "How do copy a file to a Docker container?"
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": user_question,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }, 
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

response = es.search(index=index_name, body=search_query)

In [100]:
# Q4
if "hits" in response:
    print("\nKeys in response['hits']:")
    print(response["hits"].keys())
    print("\nFirst hit example:")
    if response["hits"]["hits"]:
        pprint(response["hits"]["hits"][2])


Keys in response['hits']:
dict_keys(['total', 'max_score', 'hits'])

First hit example:
{'_id': 'tvSZHZcBdgauo6t84jn8',
 '_index': 'course-questions',
 '_score': 59.812744,
 '_source': {'course': 'machine-learning-zoomcamp',
             'question': 'How do I copy files from a different folder into '
                         'docker container’s working directory?',
             'section': '5. Deploying Machine Learning Models',
             'text': 'You can copy files from your local machine into a Docker '
                     "container using the docker cp command. Here's how to do "
                     'it:\n'
                     'In the Dockerfile, you can provide the folder containing '
                     'the files that you want to copy over. The basic syntax '
                     'is as follows:\n'
                     'COPY ["src/predict.py", "models/xgb_model.bin", '
                     '"./"]\t\t\t\t\t\t\t\t\t\t\tGopakumar Gopinathan'}}


In [131]:
def retrieve_documents(query, index_name="course-questions", max_results=3):
    es = Elasticsearch("http://localhost:9200")
    
    search_query = {
        "size": max_results,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }, 
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
        
    response = es.search(index=index_name, body=search_query)
    documents = [hit['_source'] for hit in response['hits']['hits']]
    return documents

In [132]:
user_question = "How do copy a file to a Docker container?"

response = retrieve_documents(user_question)

for doc in response:
    print(f"Section: {doc['section']}")
    print(f"Question: {doc['question']}")
    print(f"Answer: {doc['text'][:60]}...\n")

Section: 5. Deploying Machine Learning Models
Question: How do I debug a docker container?
Answer: Launch the container image in interactive mode and overridin...

Section: 5. Deploying Machine Learning Models
Question: How do I copy files from my local machine to docker container?
Answer: You can copy files from your local machine into a Docker con...

Section: 5. Deploying Machine Learning Models
Question: How do I copy files from a different folder into docker container’s working directory?
Answer: You can copy files from your local machine into a Docker con...



In [133]:
context_template = """
Q: {question}
A: {text}
""".strip()
context_result = ""

for doc in response:
    doc_str = context_template.format(**doc)
    context_result += ("\n\n" + doc_str)

context = context_result.strip()
print(context)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

In [134]:
# Q5
prompt = f"""
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {user_question}

CONTEXT:
{context}
""".strip()
pprint(prompt)
print(len(prompt))

("You're a course teaching assistant. Answer the QUESTION based on the CONTEXT "
 'from the FAQ database.\n'
 'Use only the facts from the CONTEXT when answering the QUESTION.\n'
 '\n'
 'QUESTION: How do copy a file to a Docker container?\n'
 '\n'
 'CONTEXT:\n'
 'Q: How do I debug a docker container?\n'
 'A: Launch the container image in interactive mode and overriding the '
 'entrypoint, so that it starts a bash command.\n'
 'docker run -it --entrypoint bash <image>\n'
 'If the container is already running, execute a command in the specific '
 'container:\n'
 'docker ps (find the container-id)\n'
 'docker exec -it <container-id> bash\n'
 '(Marcos MJD)\n'
 '\n'
 'Q: How do I copy files from my local machine to docker container?\n'
 'A: You can copy files from your local machine into a Docker container using '
 "the docker cp command. Here's how to do it:\n"
 'To copy a file or directory from your local machine into a running Docker '
 'container, you can use the `docker cp command`. Th

In [135]:
# Q6
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")
num_tokens = len(encoding.encode(prompt))
print(f"Number of tokens in prompt: {num_tokens}")


Number of tokens in prompt: 320


In [136]:
# bonus question 1
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "user", "content": prompt}
    ]
)

print(response.choices[0].message.content)

To copy a file to a Docker container, you can use the docker cp command. The syntax would look like this: 
docker cp /path/to/local/file_or_directory container_id:/path/in/container.


In [139]:
# bonus question 2
in_cost=0.005/1000; out_cost=0.015/1000;
cost1 =150*in_cost+250*out_cost*1000
print(f"general cost of 1000 questions in dollars: {cost1}")

cost2=len(encoding.encode(prompt))*in_cost +len(encoding.encode(response.choices[0].message.content))*out_cost
print(f"cost curent prompt in dollars: {cost2}")




general cost of 1000 questions in dollars: 3.75075
cost curent prompt in dollars: 0.002215
